<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-из-файлов" data-toc-modified-id="Изучение-данных-из-файлов-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файлов</a></span><ul class="toc-item"><li><span><a href="#Чтение-данных" data-toc-modified-id="Чтение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Чтение данных</a></span></li></ul></li><li><span><a href="#Задания" data-toc-modified-id="Задания-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задания</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Анализ базы данных книг. SQL

Компания купила крупный сервис для чтения книг по подписке. Необходимо понимать, что предлагать читателям, чтобы они были заинтересованы приложением.

**Цель исследования:** проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательских обзоров книг. Сформулировать ценностное предложение для нового продукта.


**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<a href='https://disk.yandex.ru/i/3VCf1F0YoeYLmw'> 
Структура базы данных </a>

**План исследования:** изучить полученные данные, сделать по ним запросы и проанализировать результат. 

## Изучение данных из файлов

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine
import pandas.io.sql as psql

In [2]:
# устанавление параметров
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxjyxfsf3fs.mdb.yandexcloud.net',
'port': 6462, # порт подключения
'db': 'data-analyst-final-project'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)  
          
# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Чтение данных 

In [3]:
# список таблиц
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
# вывод первых пяти строк из таблиц для ознакомления
for table in tables:
    query = f''' SELECT * 
            FROM {table}
            LIMIT 5
        '''
    print(f'Таблица {table}:')
    display(pd.io.sql.read_sql(query, con = engine)) 

Таблица books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
# какие ещё таблицы доступны в базе
query = '''
            SELECT *
            FROM pg_catalog.pg_tables
            WHERE schemaname != 'pg_catalog' AND
                  schemaname != 'information_schema';
        '''

display(pd.io.sql.read_sql(query, con = engine))

# типы столбцов в интересующих таблицах
query = '''
            SELECT 
                table_name, 
                column_name, 
                data_type, 
                is_nullable
            FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
        '''

display(pd.io.sql.read_sql(query, con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,trip,praktikum_student,None,False,False,False,False
4,public,C:oks,praktikum_student,None,False,False,False,False
5,public,boks,praktikum_student,None,False,False,False,False
6,public,finished_lessons1,praktikum_student,None,True,False,False,False
7,public,finished_lessons2,praktikum_student,None,True,False,False,False
8,public,publishers,praktikum_admin,None,True,False,True,False
9,public,authors,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,publishers,publisher_id,integer,NO
1,publishers,publisher,text,YES
2,authors,author_id,integer,NO
3,authors,author,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## Задания

In [5]:
# функция для запроса и вывода полученной информации
def query_sql(query):
    query = query
    return pd.io.sql.read_sql(query, con = engine)

1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [6]:
query_sql(''' 
             SELECT COUNT(*) 
             FROM books 
             WHERE publication_date >'2000-01-01'
          ''')    

,count
0,819


После 1 января 2000 года было выпущено 819 книг.

2. Посчитать для каждой книги количество обзоров и среднюю оценку

In [7]:
query_sql('''
             WITH avg_ratings AS (SELECT book_id, AVG(rating) AS avg_rating
                                 FROM ratings 
                                 GROUP BY book_id),
                  reviews_count AS (SELECT b.book_id, COUNT(rev.review_id) AS review_count
                                    FROM books AS b 
                                    LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
                                    GROUP BY b.book_id)
            SELECT b.book_id, b.title, rc.review_count, ROUND(ar.avg_rating, 2) AS avg_rating 
            FROM reviews_count AS rc
            INNER JOIN books AS b ON b.book_id=rc.book_id
            LEFT JOIN avg_ratings AS ar ON ar.book_id=b.book_id 
            ORDER BY rc.review_count DESC
         ''')     

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,695,The Curious Incident of the Dog in the Night-Time,6,4.08
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,696,The Da Vinci Code (Robert Langdon #2),6,3.83
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,808,The Natural Way to Draw,0,3.00
997,221,Essential Tales and Poems,0,4.00
998,387,Leonardo's Notebooks,0,4.00


Самое большое число обзоров у книги Twilight (Twilight #1), хотя ее средняя оценка всего 3,66

3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [8]:
query_sql(''' 
             SELECT p.publisher,
                    COUNT(b.book_id) AS book_count
             FROM publishers AS p
             INNER JOIN books AS b ON b.publisher_id = p.publisher_id
             WHERE b.num_pages >50
             GROUP BY p.publisher
             ORDER BY book_count DESC
             LIMIT 1
          ''')   

,publisher,book_count
0,Penguin Books,42


Самое большое количество книг толще 50 страниц было выпущено издательством Penguin Books - 42 книги.

4. Определить автора с самой высокой средней оценкой книг (рассматриваются только книги с 50 и более оценками)

In [9]:
query_sql('''
             WITH rc AS (SELECT book_id, AVG(rating)
                         FROM ratings
                         GROUP BY book_id
                         HAVING COUNT(rating)>=50),
                  a AS (SELECT a.author, b.book_id 
                        FROM authors AS a
                        LEFT JOIN books AS b ON b.author_id  = a.author_id)   
             SELECT  a.author, ROUND(AVG(rc.avg), 2) AS avg_rating 
             FROM a INNER JOIN rc ON a.book_id=rc.book_id
             GROUP BY a.author
             ORDER BY avg_rating DESC
             LIMIT 1
         ''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


Среди книг с оценками выше 50, самую высокую среднюю оценку получила J.K. Rowling/Mary GrandPré	-4,28 балла

5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [10]:
query_sql('''
             WITH ra AS (SELECT username, COUNT(rating) AS rating_count
                         FROM ratings
                         GROUP BY username),
                  r AS (SELECT username, COUNT(review_id) AS review_count
                        FROM reviews
                        GROUP BY username)
             SELECT ROUND(AVG(r.review_count), 2) AS mean_reviews
             FROM ra INNER JOIN r ON ra.username = r.username
             WHERE ra.rating_count > 50
          ''')   



,mean_reviews
0,24.33


Пользователи, которые поставили более 50 оценок, в среднем написали 24 обзора.

## Вывод

В ходе исследования и формирования запросов было установлено, что:

- после 1 января 2000 года было выпущено 819 книг
- самое большое число обзоров у книги Twilight (Twilight #1) - 7
- самое большое количество книг толще 50 страниц было выпущено издательством Penguin Books - 42 книги
- среди книг с оценками выше 50, самую высокую среднюю оценку получила J.K. Rowling/Mary GrandPré -4,28 балла
- пользователи, которые поставили более 50 оценок, в среднем написали 24 обзора.